# Sprint1課題　機械学習フロー

1.この課題の目的
機械学習の実践的な流れを知る
高い汎化性能を持つモデルを完成させる

2.機械学習フロー
1回目のSprintでは、機械学習の実践的な流れを抑えます。
このSprintはWeek3,4の延長になり、引き続きKaggleの Home Credit Default Risk コンペティションを用います。
特にここでは適切な 検証 を組み込むことを重視し、その上で前処理やモデルの選定を進めていきます。

適切な検証を行い、テストデータに対して高い汎化性能を持つモデルを完成させましょう。

In [1]:
import pandas as pd
import numpy as npa
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None

df_train=pd.read_csv('./application_train.csv')
df_test_X=pd.read_csv('./application_test.csv')

In [2]:
print(df_train.shape,df_test_X.shape)

(307511, 122) (48744, 121)


In [3]:
df_X = df_train.append(df_test_X, sort=False)

In [4]:
df_X.shape

(356255, 122)

In [5]:
# 文字列をダミー変数に置き換える        
df_X = pd.get_dummies(df_X)

In [6]:
# flaot16に揃えるとintがinfになる謎。解明は未来に持ち越す
# df_train = df_train.astype('float16')
# df_test_X = df_test_X.astype('float16')

In [7]:
# 欠損値を埋める
df_X = df_X.fillna(0)

In [8]:
df_X.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business

In [9]:
# 前処理が終わったので、結合したデータセット（train + test）を再び分割する

df_train = df_X.iloc[:df_train.shape[0],:]
df_test = df_X.iloc[df_train.shape[0]:,:]

In [10]:
# 変数を定義する

df_train_y = df_train['TARGET']
df_train_X = df_train #del ...['TARGET']するために定義

# 'TARGET'カラムを落として特徴量onlyデータを作る

del df_train_X['TARGET']
del df_test['TARGET']

### 【問題1】クロスバリデーション

事前学習期間は検証用データを分割しておき、それに対して指標値を計算することで検証を行っていました。
しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。

具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。
複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。

sklearn.model_selection.KFold — scikit-learn 0.20.2 documentation

In [11]:
#　欠損値が存在するか確認（Falseが返って来ればOK）
print(df_train_X.isnull().values.sum()!= 0)

False


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [13]:
# Kfoldを使ってn_splits回分割用のindexを生成しkfに格納

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(df_train)
print(kf.get_n_splits(df_train))

5


In [14]:
# DFをarrayに変換（DFだとkf.splitが動かないので）

ar_train_X = df_train_X.values
ar_train = df_train.values
ar_train_y = df_train_y.values

In [15]:
# indexが格納されたkfを用いてデータを分割

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

result=[]

for train_index, test_index in kf.split(ar_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = ar_train[train_index], ar_train[test_index]
    y_train, y_test = ar_train_y[train_index], ar_train_y[test_index]
    clf = RandomForestClassifier()    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
  
    auc = roc_auc_score(y_test, y_pred)
    
    result.append(auc)
    
result = np.array(result)
print(result)
print("AUC: {:.2f}+/-{:.2f}".format(result.mean(),result.std()))


TRAIN: [     0      1      2 ... 307507 307509 307510] TEST: [     3      6      9 ... 307500 307501 307508]


/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TRAIN: [     0      1      2 ... 307507 307508 307510] TEST: [     7      8     16 ... 307489 307495 307509]


/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [     4     10     12 ... 307473 307475 307476]


...

TRAIN: [     3      4      6 ... 307508 307509 307510] TEST: [     0      1      2 ... 307503 307504 307506]


/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TRAIN: [     0      1      2 ... 307506 307508 307509] TEST: [    13     14     15 ... 307505 307507 307510]


/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.50147446 0.50342825 0.50230181 0.5016347  0.50259241]
AUC: 0.50+/-0.00


### 【問題2】グリッドサーチ

これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。
パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。
パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。
グリッドサーチをパイプラインの中に組み込みましょう。

ロジスティック回帰：

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [16]:
# ハイパーパラメータ最適化を自動で行う
# あらかじめ選択した任意のハイパーパラメータ部分集合から網羅的に探索　-> GridSearchCV使用
# モデルはロジスティック回帰
# 探索ハイパーパラメータはC（正則化項）とペナルティ


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid={"C":np.logspace(-3,3,7),
            "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,param_grid,cv=2, scoring='roc_auc')
logreg_cv.fit(ar_train, ar_train_y)

print(logreg_cv.best_score_)  # 最も良かったスコア
print(logreg_cv.best_params_)

/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Nonna/.pyenv/versions/anaconda3-4.3.0/lib/python3

0.7290158416992844
{'C': 1.0, 'penalty': 'l1'}


In [17]:
# 確率にて予測値を算出

y_pred_best = logreg_cv.best_estimator_.predict_proba(df_test)
y_pred_best[:,1].shape

(48744,)

In [18]:
# sample_submissionファイルに予測値を代入
# 提出ファイルをcsvで出力

submission = pd.read_csv('sample_submission.csv')
submission['TARGET'] = y_pred_best[:,1]
submission.to_csv('pipeline_1.csv', index=False)

FileNotFoundError: File b'sample_submission.csv' does not exist

In [ ]:
submission.head()

### 【問題3】Kernelからの調査

KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。
そして、効果があると考えられるものを検証してください。

### 【問題4】高い汎化性能のモデル

In [ ]:
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。
今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。